In [1]:
! pip install boto3
! pip install selenium
! pip install awswrangler # Biblioteca p interagir com servicos da aws 


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from selenium import webdriver
from config import linkedin_username, linkedin_password
from variaveis import vaga
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import re
import awswrangler as wr

In [3]:
def login_linkedin():
    browser = webdriver.Chrome()
    browser.get('https://www.linkedin.com/login')

    campo_email = browser.find_element(By.ID, 'username')
    campo_email.send_keys(linkedin_username)
    campo_senha = browser.find_element(By.ID, 'password')
    campo_senha.send_keys(linkedin_password)
    login = browser.find_element(By.XPATH, "//button[@type='submit']")
    login.click()
    sleep(5)  # Tempo de espera após o login

    return browser

In [3]:
# Realiza a busca pela vaga de emprego
def realizar_busca_vaga(browser, vaga): 
    campo_busca_vaga = browser.find_element(By.XPATH, "//input[@placeholder='Pesquisar']")
    campo_busca_vaga.send_keys(vaga)
    campo_busca_vaga.send_keys(Keys.RETURN)
    sleep(10)

In [4]:
def aplicar_filtros(browser):
    filtro_vagas = browser.find_element(By.XPATH, "//button[text()='Vagas']")
    filtro_vagas.click()
    sleep(10)
    
    # Clicar em "Todos os filtros"
    todos_filtros = browser.find_element(By.XPATH, "//button[text()='Todos os filtros']")
    todos_filtros.click()
    sleep(8)
    
    # DATA DO ANUNCIO:
    data_anuncio = browser.find_element(By.XPATH, "//span[text()='A qualquer momento']")
    data_anuncio.click()
    sleep(3)
    
    
    # MODALIDADE DA VAGA:
    # Rolar o scroll até o elemento "Remoto" -  clica em remoto
    remoto = browser.find_element(By.XPATH, "//span[text()='Remoto']")
    remoto.click()
    sleep(3)

    presencial = browser.find_element(By.XPATH, "//span[text()='Presencial']")
    presencial.click()
    sleep(3)

    hibrido = browser.find_element(By.XPATH, "//span[text()='Híbrido']")
    hibrido.click()
    sleep(3)


    # Encontrar o botão "Exibir resultados" usando seletores CSS
    botao_exibir_resultados = browser.find_element(By.CSS_SELECTOR, "button[aria-live='polite']")
    botao_exibir_resultados.click()
    sleep(10)

In [5]:
# Rola a página para baixo para carregar mais resultados.
def rolar_pagina(browser, pixels):
    elementos = browser.find_element(By.CSS_SELECTOR, "main div.jobs-search-results-list")
    browser.execute_script(f"arguments[0].scrollTop += {pixels};", elementos)
    sleep(2)


In [6]:
# Obtém os links das vagas encontradas.
def obter_links_vagas(browser):
    links = []
    for _ in range(25):
        rolar_pagina(browser, 200)
        links = browser.find_elements(By.XPATH, "//main//div/div//ul//li//a[@data-control-id]")
        print(len(links))
        if len(links) >= 25:
            print(f'chegamos ao numero esperado de {len(links)}')
            break
    return links


In [7]:
# Navega para a próxima página de vagas.
def proxima_pag_vagas(browser, numero_pagina):
    try:
        pagina_vaga = browser.find_element(By.XPATH, f"//button[@aria-label='Página {numero_pagina}']")
        pagina_vaga.click()
        sleep(5)
        print(f"Clicou na página {numero_pagina} com sucesso!")
    except NoSuchElementException:
        print(f"NoSuchElementException: Não encontrou o botão para a página {numero_pagina}.")
       
       

In [8]:
def extrair_dados_vagas(browser, links): 
    dados = []
    for link in links:
        nome_vaga = link.text.split('\n')[0]    # .text é um método que retorna o texto que é exibido na tela para o usuário. split: Pegue apenas a primeira linha
        url_link = link.get_attribute("href") # atribui o URL do link à variável url_link. href é o elemento. 

        # Localizar nome da empresa e localização
        try:
            container = link.find_element(By.XPATH, "../..")
            empresa = container.find_element(By.CLASS_NAME, 'job-card-container__primary-description').text.strip()
            localizacao = container.find_element(By.CLASS_NAME, 'job-card-container__metadata-wrapper').text.strip()

        except NoSuchElementException:
            empresa = "Empresa não disponível"
            localizacao = "Localização não disponível"

       # Adiciona informações sobre a vaga em um dicionário e adiciona esse dicionário à lista de dados.
        dados.append({
            'nome_vaga': nome_vaga, 
            'empresa': empresa, 
            'localizacao': localizacao, 
            'link_vaga': url_link, 
            
        })

    return dados


In [9]:

# Encerra o navegador
def encerrar_browser(browser):  
    print("Encerrando busca...")
    print("Arquivo CSV salvo com sucesso!")
    sleep(3)
    browser.quit()

In [10]:
if __name__ == "__main__":
    # Início do processo
    browser = login_linkedin()
    realizar_busca_vaga(browser, vaga)
    aplicar_filtros(browser)

    # Criar DataFrame para armazenar os dados
    colunas = ['nome_vaga', 'empresa', 'localizacao', 'link_vaga']
    df = pd.DataFrame(columns=colunas)

    # Extrair informações da página atual
    links = obter_links_vagas(browser)
    dados = extrair_dados_vagas(browser, links)
    df = pd.concat([df, pd.DataFrame(dados)], ignore_index=True)

    # Configurar variáveis para o loop
    condicao = True
    numero_pagina = 2

    while condicao:
        proxima_pag_vagas(browser, str(numero_pagina))
        # Verifica se ainda há mais páginas disponíveis
        try:
            browser.find_element(By.XPATH, f"//button[@aria-label='Página {numero_pagina + 1}']")
        except NoSuchElementException:
            print("Não há mais páginas disponíveis. Encerrando...")
            condicao = False
    
      # Extrair informações
        links = obter_links_vagas(browser)
        dados = extrair_dados_vagas(browser, links)
        df = pd.concat([df, pd.DataFrame(dados)], ignore_index=True)
    
        numero_pagina += 1

    # Salvar DataFrame como arquivo CSV
    df.to_csv('vagas_linkedin.csv', index=False)

    # Encerrar o navegador
    encerrar_browser(browser)

7
7
7
9
11
12
14
16
17
19
21
22
24
25
chegamos ao numero esperado de 25
Clicou na página 2 com sucesso!
7
7
8
10
12
13
15
16
18
20
21
23
25
chegamos ao numero esperado de 25
Clicou na página 3 com sucesso!
7
7
8
10
12
13
15
17
18
20
22
23
25
chegamos ao numero esperado de 25
Clicou na página 4 com sucesso!
7
7
7
9
10
12
14
16
17
19
20
22
24
25
chegamos ao numero esperado de 25
Clicou na página 5 com sucesso!
7
7
8
9
11
12
14
15
17
19
21
22
24
25
chegamos ao numero esperado de 25
Clicou na página 6 com sucesso!
7
7
8
9
11
12
14
16
18
19
21
22
24
25
chegamos ao numero esperado de 25
Clicou na página 7 com sucesso!
7
7
8
10
12
13
15
17
18
20
21
23
25
chegamos ao numero esperado de 25
Clicou na página 8 com sucesso!
7
7
8
10
12
13
15
17
18
20
21
23
24
25
chegamos ao numero esperado de 25
Clicou na página 9 com sucesso!
7
7
8
10
12
14
15
17
18
20
22
24
25
chegamos ao numero esperado de 25
Clicou na página 10 com sucesso!
7
7
8
9
11
13
14
16
17
19
21
22
24
25
chegamos ao numero esperado de 2

In [11]:
dados_vagas_linkedin = pd.read_csv("/Users/mac/MeuProjeto/vagas_linkedin.csv")
df = pd.DataFrame(dados_vagas_linkedin)

num_linhas = df.shape[0]
print("Número de linhas:", num_linhas)


Número de linhas: 998


In [12]:
def categorizar_cargo(nome_vaga):
    cargo = nome_vaga.lower()
    if re.search(r'\b(s[eê]nior|sr\.|sênior|senior|engenheiro de dados sr|engenheiro de dados s[êe]nior|data engineer sr)\b', cargo):
        return 'Senior'
    elif re.search(r'\b(pleno|pl\.|pleno|ii|engenheiro de dados pl|data engineer pl)\b', cargo):
        return 'Pl'
    elif re.search(r'\b(j[uú]nior|jr\.|engenheiro de dados j[uú]nior)\b', cargo):
        return 'Junior'
    elif re.search(r'\b(engenheiro de dados iii)\b', cargo):
        return 'Senior'
    else:
        return 'nao especificado'

In [13]:
def extract_params(df):    
    # Dividir o DataFrame com base na categoria
    df_senior = df[df['Categoria'] == 'Senior']
    df_pl = df[df['Categoria'] == 'Pl']
    df_junior = df[df['Categoria'] == 'Junior']
    df_nao_especificado = df[df['Categoria'] == 'nao especificado']
    
    return df_senior,df_pl,df_junior,df_nao_especificado

In [14]:
def main():
    print(df.columns)
    df['Categoria'] = df['nome_vaga'].apply(categorizar_cargo)

    df_senior,df_pl,df_junior,df_outras_vagas=extract_params(df)

    df_join = pd.concat([df_senior,df_pl,df_junior,df_outras_vagas])
    
    wr.s3.to_parquet(
        df=pd.DataFrame(df_join),
        path='s3://my-gabizinha-bkt-2/linkedin_input/tb_vagas_linkedin/anomesdia=20240428/file_20240428.parquet'
    )
    
if __name__ == "__main__":
    main()

Index(['nome_vaga', 'empresa', 'localizacao', 'link_vaga'], dtype='object')
